## Step 1: Model Size Reference

In [ ]:
print("="*70)
print("📊 MODEL SIZE REFERENCE FOR DUAL T4 (30GB)")
print("="*70)

models_table = """
┌──────────────────┬────────────────┬──────────────────┬──────────────────┐
│ Model Size       │ Quantization   │ VRAM Needed      │ Fits Dual T4?    │
├──────────────────┼────────────────┼──────────────────┼──────────────────┤
│ 1-3B             │ Q4_K_M         │ ~2-3 GB          │ ✅ Single GPU    │
│ 7-8B             │ Q4_K_M         │ ~5-6 GB          │ ✅ Single GPU    │
│ 7-8B             │ F16            │ ~15 GB           │ ✅ Single GPU    │
│ 13B              │ Q4_K_M         │ ~8-9 GB          │ ✅ Single GPU    │
│ 13B              │ Q8_0           │ ~14-15 GB        │ ✅ Single GPU    │
│ 32-34B           │ Q4_K_M         │ ~20-22 GB        │ ✅ Dual GPU      │
│ 70B              │ Q4_K_M         │ ~40-42 GB        │ ❌ Too Large     │
│ 70B              │ IQ3_XS         │ ~25-27 GB        │ ✅ Dual GPU      │
│ 70B              │ IQ2_XXS        │ ~20-22 GB        │ ✅ Dual GPU      │
└──────────────────┴────────────────┴──────────────────┴──────────────────┘

📝 Recommended Models for Kaggle:
• Best Quality (Single T4):  7-8B Q8_0 or F16
• Best Quality (Dual T4):    32B Q4_K_M
• Largest Possible:          70B IQ3_XS (aggressive quantization)
"""
print(models_table)

## Step 2: I-Quant Types for Large Models

In [ ]:
print("="*70)
print("🔢 I-QUANT TYPES (FOR 70B+ MODELS)")
print("="*70)

iquant_table = """
┌──────────────────┬────────────────┬──────────────────┬──────────────────┐
│ Quantization     │ Bits/Weight    │ 70B VRAM         │ Quality          │
├──────────────────┼────────────────┼──────────────────┼──────────────────┤
│ IQ4_XS           │ ~4.25 bits     │ ~35 GB           │ ⭐⭐⭐⭐         │
│ IQ3_M            │ ~3.4 bits      │ ~28 GB           │ ⭐⭐⭐⭐         │
│ IQ3_S            │ ~3.0 bits      │ ~26 GB           │ ⭐⭐⭐           │
│ IQ3_XS           │ ~3.0 bits      │ ~25 GB ✓         │ ⭐⭐⭐           │
│ IQ3_XXS          │ ~2.8 bits      │ ~24 GB ✓         │ ⭐⭐⭐           │
│ IQ2_M            │ ~2.7 bits      │ ~23 GB ✓         │ ⭐⭐             │
│ IQ2_S            │ ~2.5 bits      │ ~21 GB ✓         │ ⭐⭐             │
│ IQ2_XS           │ ~2.3 bits      │ ~20 GB ✓         │ ⭐⭐             │
│ IQ2_XXS          │ ~2.0 bits      │ ~18 GB ✓         │ ⭐               │
│ IQ1_M            │ ~1.75 bits     │ ~16 GB ✓         │ ⭐               │
│ IQ1_S            │ ~1.5 bits      │ ~14 GB ✓         │ Experimental     │
└──────────────────┴────────────────┴──────────────────┴──────────────────┘

📝 Recommendation for 70B on Dual T4:
   ✓ IQ3_XS  - Best balance of quality and size
   ✓ IQ2_XXS - Fits comfortably, lower quality
"""
print(iquant_table)

## Step 3: Setup Environment

In [ ]:
%%time
# Install llcuda v2.2.0 (force fresh install to ensure correct binaries)
!pip install -q --no-cache-dir --force-reinstall git+https://github.com/llcuda/llcuda.git@v2.2.0
!pip install -q huggingface_hub

import llcuda
print(f"✅ llcuda {llcuda.__version__} installed")

# Verify GPUs
!nvidia-smi --query-gpu=index,name,memory.total,memory.free --format=csv

## Step 4: Download 70B IQ3_XS Model

In [ ]:
%%time
from huggingface_hub import hf_hub_download
import os

print("="*70)
print("📥 DOWNLOADING 70B MODEL (IQ3_XS)")
print("="*70)

# Example: Llama 2 70B IQ3_XS
# Note: Replace with actual model repo as needed
MODEL_REPO = "TheBloke/Llama-2-70B-GGUF"  # Example repo
MODEL_FILE = "llama-2-70b.IQ3_XS.gguf"    # ~25GB file

print(f"📦 Downloading {MODEL_FILE}...")
print("⏳ This may take 15-30 minutes on Kaggle...\n")

try:
    model_path = hf_hub_download(
        repo_id=MODEL_REPO,
        filename=MODEL_FILE,
        local_dir="/kaggle/working/models",
        resume_download=True  # Resume if interrupted
    )
    
    file_size = os.path.getsize(model_path) / 1024**3
    print(f"\n✅ Downloaded: {model_path}")
    print(f"📊 Size: {file_size:.2f} GB")
    
except Exception as e:
    print(f"⚠️ Download failed: {e}")
    print("\n📝 Alternative: Use a smaller 70B quant or different model")
    print("   Example repos with 70B IQ3_XS:")
    print("   • bartowski/Llama-3.1-70B-Instruct-GGUF")
    print("   • bartowski/Qwen2.5-72B-Instruct-GGUF")

## Step 5: Calculate Optimal Tensor Split

In [ ]:
print("="*70)
print("⚙️ CALCULATING TENSOR SPLIT")
print("="*70)

import torch

# Get available memory on each GPU
gpu0_free = torch.cuda.get_device_properties(0).total_memory / 1024**3
gpu1_free = torch.cuda.get_device_properties(1).total_memory / 1024**3

print(f"\n📊 GPU 0: {gpu0_free:.1f} GB total")
print(f"📊 GPU 1: {gpu1_free:.1f} GB total")

# For identical GPUs, use 50/50 split
total = gpu0_free + gpu1_free
split0 = gpu0_free / total
split1 = gpu1_free / total

print(f"\n📊 Calculated tensor split: {split0:.2f},{split1:.2f}")

# Leave headroom for KV cache
TENSOR_SPLIT = "0.48,0.48"  # Leave ~1GB per GPU for KV cache
print(f"📊 Recommended split (with headroom): {TENSOR_SPLIT}")

## Step 6: Start Server with 70B Model

In [ ]:
%%time
from llcuda.server import ServerManager

print("="*70)
print("🚀 STARTING 70B MODEL ON DUAL T4")
print("="*70)

# Configuration for 70B on dual T4
print("\n📊 Server Configuration:")
print(f"   Model: 70B IQ3_XS")
print(f"   Tensor Split: {TENSOR_SPLIT}")
print(f"   Context: 2048")
print(f"   Flash Attention: True")

server = ServerManager()
server.start_server(
    model_path=model_path,  # From download step
    host="127.0.0.1",
    port=8080,
    
    # GPU Configuration
    gpu_layers=999,           # All layers on GPU
    tensor_split=TENSOR_SPLIT,  # Distribute across both T4s
    
    # Memory Optimization
    ctx_size=2048,          # Smaller context for 70B
    flash_attention=True,            # Enable flash attention
    
    # Performance
    batch_size=128,                # Smaller batch for memory
)

print("\n⏳ Loading model (may take 2-5 minutes for 70B)...")
if server.check_server_health(timeout=300):  # 5 minute timeout
    print("\n✅ 70B model loaded on dual T4!")
else:
    print("\n❌ Server failed to start")

## Step 7: Verify GPU Memory Usage

In [ ]:
print("="*70)
print("💾 GPU MEMORY USAGE")
print("="*70)

!nvidia-smi

print("\n📊 Expected memory distribution:")
print("   GPU 0: ~12-13 GB (model layers 0-39)")
print("   GPU 1: ~12-13 GB (model layers 40-79)")

## Step 8: Test 70B Inference

In [ ]:
from llcuda.api.client import LlamaCppClient
import time

print("="*70)
print("🧪 TESTING 70B INFERENCE")
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8080")

# Test prompt
prompt = "Explain quantum computing in simple terms."

print(f"\n💬 Prompt: {prompt}")
print("\n📝 Response:\n")

start = time.time()

response = client.chat_completion(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=200,
    temperature=0.7,
)

elapsed = time.time() - start
tokens = response.usage.completion_tokens

print(response.choices[0].message.content)

print(f"\n📊 Performance:")
print(f"   Time: {elapsed:.2f}s")
print(f"   Tokens: {tokens}")
print(f"   Speed: {tokens/elapsed:.1f} tok/s")

## Step 9: Performance Benchmarks

In [ ]:
import time

print("="*70)
print("📊 PERFORMANCE BENCHMARKS")
print("="*70)

prompts = [
    "Hello",  # Very short
    "What is the capital of France?",  # Short
    "Write a paragraph about machine learning.",  # Medium
    "Explain the differences between Python and JavaScript with examples.",  # Long
]

print("\n🏃 Running benchmarks...\n")

for prompt in prompts:
    start = time.time()
    
    response = client.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7,
    )
    
    elapsed = time.time() - start
    tokens = response.usage.completion_tokens
    
    print(f"   Prompt: \"{prompt[:40]}...\"")
    print(f"   Time: {elapsed:.2f}s | Tokens: {tokens} | Speed: {tokens/elapsed:.1f} tok/s")
    print()

print("""
📝 Expected Performance (70B on Dual T4):
   • IQ3_XS: ~5-10 tok/s
   • IQ2_XXS: ~8-15 tok/s (lower quality)
   
   Compared to:
   • 7B Q4_K_M on single T4: ~40-60 tok/s
   • 13B Q4_K_M on single T4: ~25-35 tok/s
""")

## Step 10: Alternative - 32B Model (Better Speed)

In [ ]:
print("="*70)
print("⚡ ALTERNATIVE: 32B MODEL FOR BETTER SPEED")
print("="*70)

print("""
📝 If 70B is too slow, consider 32B models:

┌──────────────────────────────────────────────────────────────────────┐
│ Model                      │ Quantization │ VRAM     │ Speed        │
├────────────────────────────┼──────────────┼──────────┼──────────────┤
│ Qwen2.5-32B-Instruct      │ Q4_K_M       │ ~20 GB   │ ~15-25 tok/s │
│ DeepSeek-V2-Lite-33B      │ Q4_K_M       │ ~20 GB   │ ~15-25 tok/s │
│ Yi-34B                     │ Q4_K_M       │ ~21 GB   │ ~15-25 tok/s │
│ Mixtral-8x7B (MoE)        │ Q4_K_M       │ ~26 GB   │ ~20-30 tok/s │
└──────────────────────────────────────────────────────────────────────┘

📦 Example download:

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="bartowski/Qwen2.5-32B-Instruct-GGUF",
    filename="Qwen2.5-32B-Instruct-Q4_K_M.gguf",
    local_dir="/kaggle/working/models"
)
""")

## Step 11: Streaming for Better UX

In [ ]:
print("="*70)
print("🌊 STREAMING WITH LARGE MODELS")
print("="*70)

print("\n💬 Streaming response (feels faster!):\n")

start = time.time()
token_count = 0

for chunk in client.chat_completion_stream(
    messages=[{"role": "user", "content": "What are the benefits of streaming?"}],
    max_tokens=100,
    temperature=0.7,
):
    if chunk.choices and chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        print(content, end="", flush=True)
        token_count += 1

elapsed = time.time() - start
print(f"\n\n📊 Streamed {token_count} tokens in {elapsed:.2f}s")
print("\n✅ Streaming provides immediate feedback even with slower models!")

## Step 12: Memory Tips

In [ ]:
print("="*70)
print("💡 MEMORY OPTIMIZATION TIPS")
print("="*70)

print("""
🔹 Context Size Impact:

   Context   │ KV Cache per GPU │ Recommended for 70B
   ──────────┼──────────────────┼─────────────────────
   512       │ ~0.5 GB          │ ✅ Safe
   2048      │ ~1.5 GB          │ ✅ Default
   4096      │ ~3 GB            │ ⚠️ Tight fit
   8192      │ ~6 GB            │ ❌ May OOM

🔹 Batch Size Impact:

   n_batch │ Memory │ Speed
   ────────┼────────┼──────
   64      │ Lower  │ Slower
   128     │ Medium │ Medium (recommended)
   256     │ Higher │ Faster
   512     │ High   │ Fastest (may OOM)

🔹 If Running Out of Memory:

   1. Reduce context_size (2048 → 1024)
   2. Reduce n_batch (128 → 64)
   3. Use smaller quantization (IQ3_XS → IQ2_XXS)
   4. Keep some layers on CPU (n_gpu_layers=60)
""")

## Step 13: Cleanup

In [ ]:
print("🛑 Stopping server...")
server.stop_server()
print("✅ Server stopped")

# Check memory freed
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

## 📚 Summary

### Model Recommendations for Kaggle Dual T4:

| Goal | Model | Quantization | VRAM | Speed |
|------|-------|-------------|------|-------|
| Best Speed | 7-8B | Q4_K_M | 5GB | 50+ tok/s |
| Best Balance | 13B | Q4_K_M | 8GB | 30+ tok/s |
| Best Quality | 32B | Q4_K_M | 20GB | 20+ tok/s |
| Largest | 70B | IQ3_XS | 25GB | 8-10 tok/s |

### Key Configuration for 70B:
```python
ServerConfig(
    n_gpu_layers=999,
    tensor_split="0.48,0.48",
    context_size=2048,  # Keep small
    flash_attn=True,
    n_batch=128,        # Keep moderate
)
```

### I-Quant Selection:
- **IQ3_XS** - Best quality that fits dual T4
- **IQ2_XXS** - More headroom, lower quality

---

**Next:** [10-complete-workflow](10-complete-workflow-llcuda-v2.2.0.ipynb)